In [ ]:
# 1. Load data bureau
import pandas as pd
import os

print(os.listdir())  

# Load file bureau.csv
bureau = pd.read_csv('bureau.csv')
print("Ukuran data bureau:", bureau.shape)
bureau.head()

['application_train.csv', 'bureau.csv', 'eda_application_train.ipynb', 'feature_engineer_bureau.ipynb']
Ukuran data bureau: (1716428, 17)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [13]:
# 2. Agregasi fitur numerik
bureau_agg = bureau.groupby('SK_ID_CURR').agg({
    'AMT_CREDIT_SUM': ['mean', 'sum'],
    'DAYS_CREDIT': ['mean', 'min'],
    'CREDIT_DAY_OVERDUE': ['mean', 'max'],
    'CNT_CREDIT_PROLONG': ['sum'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],
    'AMT_ANNUITY': ['mean']
})

In [14]:
# 3. Rename kolom multi-index
bureau_agg.columns = ['BUREAU_' + '_'.join(col).upper() for col in bureau_agg.columns]
bureau_agg.reset_index(inplace=True)

print("Contoh fitur agregat bureau:")
display(bureau_agg.head())

Contoh fitur agregat bureau:


,SK_ID_CURR,BUREAU_AMT_CREDIT_SUM_MEAN,BUREAU_AMT_CREDIT_SUM_SUM,BUREAU_DAYS_CREDIT_MEAN,BUREAU_DAYS_CREDIT_MIN,BUREAU_CREDIT_DAY_OVERDUE_MEAN,BUREAU_CREDIT_DAY_OVERDUE_MAX,BUREAU_CNT_CREDIT_PROLONG_SUM,BUREAU_AMT_CREDIT_SUM_DEBT_MEAN,BUREAU_AMT_CREDIT_SUM_DEBT_SUM,BUREAU_AMT_ANNUITY_MEAN
0,100001,207623.571429,1453365.000,-735.000000,-1572,0.0,0,0,85240.928571,596686.5,3545.357143
1,100002,108131.945625,865055.565,-874.000000,-1437,0.0,0,0,49156.200000,245781.0,0.000000
2,100003,254350.125000,1017400.500,-1400.750000,-2586,0.0,0,0,0.000000,0.0,NaN
3,100004,94518.900000,189037.800,-867.000000,-1326,0.0,0,0,0.000000,0.0,NaN
4,100005,219042.000000,657126.000,-190.666667,-373,0.0,0,0,189469.500000,568408.5,1420.500000


In [16]:
df = pd.read_csv('application_train.csv')  # atau 'data/application_train.csv' kalau filenya di folder 'data'

In [17]:
# Gabung ke df utama (application_train)
df = df.merge(bureau_agg, on='SK_ID_CURR', how='left')

print("Ukuran data setelah merge:", df.shape)

Ukuran data setelah merge: (307511, 132)


In [18]:
# Korelasi fitur baru dengan TARGET
new_corr = df.corr(numeric_only=True)['TARGET'].sort_values(ascending=False)
print("Fitur baru dari bureau.csv yang korelasinya paling tinggi:")
print(new_corr.filter(like='BUREAU_').head(10))

Fitur baru dari bureau.csv yang korelasinya paling tinggi:
BUREAU_DAYS_CREDIT_MEAN           0.089729
BUREAU_DAYS_CREDIT_MIN            0.075248
AMT_REQ_CREDIT_BUREAU_YEAR        0.019930
BUREAU_CREDIT_DAY_OVERDUE_MEAN    0.008118
BUREAU_AMT_CREDIT_SUM_DEBT_SUM    0.007144
BUREAU_CREDIT_DAY_OVERDUE_MAX     0.005493
BUREAU_CNT_CREDIT_PROLONG_SUM     0.004058
AMT_REQ_CREDIT_BUREAU_DAY         0.002704
AMT_REQ_CREDIT_BUREAU_HOUR        0.000930
AMT_REQ_CREDIT_BUREAU_WEEK        0.000788
Name: TARGET, dtype: float64


In [19]:
df.to_csv('df_merged_application_bureau.csv', index=False)